# Building your Recurrent Neural Network - Step by Step

Welcome to Course 5's first assignment! In this assignment, you will implement your first Recurrent Neural Network in numpy.

Recurrent Neural Networks (RNN) are very effective for Natural Language Processing (NLP) and other sequence tasks because they have the "memory". They can read inputs $x^{<t>}$ (such as words) on at a time and remember some information through the hidden layer activations that get passed from one-time step to the next. This allows uni-directional RNN to take information from the past and to process later inputs. A bi-directional RNN can take the context from both past and future. 

**Notation**:
+ Superscript $[l]$ denotes an object associated with the $l^{th}$ layer.
    + Eg: $a^{[4]}$ is the $4^{th}$ layer activation. $W^{[5]}$ and $b^{[5]}$ are $5^{th}$ layer parameters.
+ Superscript $(i)$ denotes an object associated with the $i^{th}$ example.
    + Eg: $x^{(i)}$ is the $i^{th}$ training example.
+ Superscript $<t>$ denotes an object at the $t^{th}$ time-step.
    + Eg: $x^{<t>}$ is the input $x$ at the $t^{th}$ time-step. $x^{(i)<t>}$ is the input at $t^{th}$ time-step of example $i$.
+ Lowerscript $i$ denotes the $i^{th}$ entry of the activations in the layer $l$.
    + Eg: $a_{i}^{[l]}$ denotes the $i^{th}$ entry of the activations in layer $l$.

In [1]:
import numpy as np

## 1. Forward Propagation for the Basic Recurrent Neural Network

Later this week, you will generate music using an RNN. The basic RNN that you will implement has the structure below. In this example, $T_x = T_y$. 

<img src="../images/RNN.png" style="width:500;height:300px;">
<caption><center> **Figure 1**: Basic RNN model </center></caption>

In [8]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def sigmoid(x):
    return 1. / (1 + np.exp(-x))

Here is how you can implement an RNN.

**Steps**:
1. Implement the calculations needed for one time-step of the RNN.
2. Implement a loop over $T_x$ time-steps in-order to process all the inputs, one at a time.

Let's go

### 1.1. RNN cell
A Recurrent neural network can be seen as the repetition of a single cell. We will first implement the computations of the single time-step. The following figure describes the operations for a single time-step of an RNN cell.

<img src="../images/rnn_step_forward.png" style="width:700px;height:300px;">
<caption><center> **Figure 2**: Basic RNN cell. Takes as input $x^{\langle t \rangle}$ (current input) and $a^{\langle t - 1\rangle}$ (previous hidden state containing information from the past), and outputs $a^{\langle t \rangle}$ which is given to the next RNN cell and also used to predict $y^{\langle t \rangle}$ </center></caption>

**Exercise**: Implement the RNN-cell described in figure-2
1. Compute the hidden state with $\text{tanh}$ activation: $a^{\langle t \rangle} = \text{tanh}(W_{ax}x^{\langle t \rangle} +  W_{aa}a^{\langle t-1 \rangle} + b_{a})$ 
2. Using the new hidden state $a^{\langle t \rangle}$, compute the prediction $\hat{y}^{\langle t \rangle} = \text{softmax}(W_{ya}a^{\langle t \rangle} + b_{y})$
3. Store $(a^{\langle t \rangle}, a^{\langle t-1 \rangle}, x^{\langle t \rangle}, \text{parameters})$ in cache.
4. Return $a^{\langle t \rangle}, y^{\langle t \rangle}$ and cache

We will vectorize over $m$ examples. Thus, $x^{\langle t \rangle}$ will have dimension $(n_x, m)$ and $a^{\langle t \rangle}$ will have dimension $(n_a, m)$.

In [3]:
def rnn_cell_forward(xt, a_prev, parameters):
    """
    Implements a single step of a RNN cell as described in Figure-2
    
    Arguments:
    ----------
    xt: np.array - (n_x, m), input data at timestep 't'
    a_prev: np.array - (n_a, m), hidden state at timestep 't-1'
    parameters: dict, 
                Wax: np.array - (n_a, n_x), weight matrix multiplying the input
                Waa: np.array - (n_a, n_a), weight matrix multiplying the hidden state
                Wya: np.array - (n_y, n_a), weight matrix relating the hidden state to output
                ba: np.array - (n_a, 1), bias
                by: np.array - (n_y, 1), bias relating the hidden-state to the output
    Returns:
    --------
    a_next: np.array - (n_a, m), next hidden state
    yt_pred: np.array - (n_y, m), prediction at timestep 't'
    cache: tuple - (a_next, a_prev, xt, parameters), values needed for the backward pass
    """
    # Retrieve parameters from "parameters"
    Wax = parameters['Wax']
    Waa = parameters['Waa']
    Wya = parameters['Wya']
    ba = parameters['ba']
    by = parameters['by']
    
    # compute next activation state
    a_next = np.tanh(np.dot(Wax, xt) + np.dot(Waa, a_prev) + ba)
    # compute output of the current cell
    yt_pred = softmax(np.dot(Wya, a_next) + by)
    # store values you need for backward propagation in cache
    cache = (a_next, a_prev, xt, parameters)
    
    return a_next, yt_pred, cache

In [4]:
np.random.seed(1)
xt = np.random.randn(3,10)
a_prev = np.random.randn(5,10)
Waa = np.random.randn(5,5)
Wax = np.random.randn(5,3)
Wya = np.random.randn(2,5)
ba = np.random.randn(5,1)
by = np.random.randn(2,1)
parameters = {"Waa": Waa, "Wax": Wax, "Wya": Wya, "ba": ba, "by": by}

a_next, yt_pred, cache = rnn_cell_forward(xt, a_prev, parameters)
print("a_next[4] = ", a_next[4])
print("a_next.shape = ", a_next.shape)
print("yt_pred[1] =", yt_pred[1])
print("yt_pred.shape = ", yt_pred.shape)

a_next[4] =  [ 0.59584544  0.18141802  0.61311866  0.99808218  0.85016201  0.99980978
 -0.18887155  0.99815551  0.6531151   0.82872037]
a_next.shape =  (5, 10)
yt_pred[1] = [0.9888161  0.01682021 0.21140899 0.36817467 0.98988387 0.88945212
 0.36920224 0.9966312  0.9982559  0.17746526]
yt_pred.shape =  (2, 10)


### 1.2. RNN Forward Pass

You can see an RNN as the repetition of the cell you've just built. If your input sequence of data is carried over 10 time steps, then you will copy the RNN cell 10 times. Each cell takes as input the hidden state from the previous cell ($a^{\langle t-1 \rangle}$) and the current time-step's input data ($x^{\langle t \rangle}$). It outputs a hidden state ($a^{\langle t \rangle}$) and a prediction ($y^{\langle t \rangle}$) for this time-step.

<img src="../images/rnn1.png" style="width:800px;height:300px;">
<caption><center> **Figure 3**: Basic RNN. The input sequence $x = (x^{\langle 1 \rangle}, x^{\langle 2 \rangle}, ..., x^{\langle T_x \rangle})$  is carried over $T_x$ time steps. The network outputs $y = (y^{\langle 1 \rangle}, y^{\langle 2 \rangle}, ..., y^{\langle T_x \rangle})$. </center></caption>



**Exercise**: Code the forward propagation of the RNN described in Figure (3).

**Instructions**:
1. Create a vector of zeros ($a$) that will store all the hidden states computed by the RNN.
2. Initialize the "next" hidden state as $a_0$ (initial hidden state).
3. Start looping over each time step, your incremental index is $t$ :
    - Update the "next" hidden state and the cache by running `rnn_cell_forward`
    - Store the "next" hidden state in $a$ ($t^{th}$ position) 
    - Store the prediction in y
    - Add the cache to the list of caches
4. Return $a$, $y$ and caches

In [5]:
def rnn_forward(x, a0, parameters):
    """
    Implement the forward propagation of the recurrent neural network described in figure-3.
    
    Arguments:
    ----------
    x: np.array - (n_x, m, T_x), input data for every timestep.
    a0: np.array - (n_a, m), initial hidden state
    paramters: dict,
                Waa: np.array - (n_a, n_a), weight matrix multiplying the hidden-state
                Wax: np.array - (n_a, n_x), weight matrix multiplying the input
                Wya: np.array - (n_y, n_a), weight matrix relating the hidden-state to output
                ba: np.array - (n_a, 1), bias
                by: np.array - (n_y, 1), bias
    Returns:
    --------
    a: np.array - (n_a, m, T_x), hidden states for every timestep
    y_pred: np.array - (n_y, m, T_x), predictions for every timestep
    caches: tuple, values needed for backward pass
    """
    caches = []
    
    # retrieves dimensions from shapes of x and Wya
    n_x, m, T_x = x.shape
    n_y, n_a = parameters['Wya'].shape
    
    # initialize a and y with zeros
    a = np.zeros((n_a, m, T_x))
    y_pred = np.zeros((n_y, m, T_x))
    
    # initialize a_next
    a_next = a0
    
    for t in range(T_x):
        # Update the next hidden state, compute the prediction, get the cache
        a_next, yt_pred, cache = rnn_cell_forward(x[:, :, t], a_next, parameters)
        # save the value for new next hidden state in a
        a[:, :, t] = a_next
        # save the value of the prediction in y
        y_pred[:, :, t] = yt_pred
        # append cache to caches
        caches.append(cache)
        
    # store values needed for backward propagation in cache
    caches = (caches, x)
    return a, y_pred, caches 

In [6]:
np.random.seed(1)
x = np.random.randn(3,10,4)
a0 = np.random.randn(5,10)
Waa = np.random.randn(5,5)
Wax = np.random.randn(5,3)
Wya = np.random.randn(2,5)
ba = np.random.randn(5,1)
by = np.random.randn(2,1)
parameters = {"Waa": Waa, "Wax": Wax, "Wya": Wya, "ba": ba, "by": by}

a, y_pred, caches = rnn_forward(x, a0, parameters)
print("a[4][1] = ", a[4][1])
print("a.shape = ", a.shape)
print("y_pred[1][3] =", y_pred[1][3])
print("y_pred.shape = ", y_pred.shape)
print("caches[1][1][3] =", caches[1][1][3])
print("len(caches) = ", len(caches))

a[4][1] =  [-0.99999375  0.77911235 -0.99861469 -0.99833267]
a.shape =  (5, 10, 4)
y_pred[1][3] = [0.79560373 0.86224861 0.11118257 0.81515947]
y_pred.shape =  (2, 10, 4)
caches[1][1][3] = [-1.1425182  -0.34934272 -0.20889423  0.58662319]
len(caches) =  2


Congratulations! You've successfully built the forward propagation of a recurrent neural network from scratch. This will work well enough for some applications, but it suffers from vanishing gradient problems. So it works best when each output $y^{\langle t \rangle}$ can be estimated using mainly "local" context (meaning information from inputs $x^{\langle t' \rangle}$ where $t'$ is not too far from $t$). 

In the next part, you will build a more complex LSTM model, which is better at addressing vanishing gradients. The LSTM will be better able to remember a piece of information and keep it saved for many timesteps. 

## 2. Long Short-Term Memory (LSTM) network

This follow up figure shows the operations of an LSTM-cell.

<img src="../images/LSTM.png" style="width:500;height:400px;">
<caption><center> **Figure 4**: LSTM-cell. This tracks and updates a "cell state" or memory variable $c^{\langle t \rangle}$ at every time-step, which can be different from $a^{\langle t \rangle}$. </center></caption>

Similar to the RNN example, you will start by implementing the LSTM cell for a single time-step. Then you can iteratively call it from inside a for-loop to have it process an input with $T_x$ time-steps.

## About the gates
+ **Forget gate**: <br/> 
For the sake of this illustration, lets assume we are reading words in piece of text, and want to use an LSTM to keep track of grammatical structures, such as subject is singular/plural. If the subject changes from singular word to a plural word, we need to find a way to get rid of our previously stored memory value of the singular/plural state. In an LSTM, the forget gate lets us do this.<br/><br/>
$$\Gamma_{f}^{\langle t \rangle} = \sigma(W_{f}[a^{\langle t-1 \rangle}, x^{\langle t \rangle}] + b_{f})$$<br>
Here $W_{f}$ are the weights that governs the forget gate's behavior. We concatenate $[a^{\langle t-1 \rangle}, x^{\langle t \rangle}]$ and multiply by $W_{f}$. The equation above results in a vector $\Gamma_{f}^{\langle t \rangle}$ with the values between 0 and 1. This forget gate vector is multiplied element-wise by the previous cell state $c^{\langle t-1 \rangle}$. So if one of the values of this forget gate $\Gamma_{f}^{\langle t \rangle}$ is zero (or close to zero) that means the LSTM should remove that piece of information (eg. the singular subject) in the corresponding component of $c^{\langle t-1 \rangle}$. If one of the values is 1, then it will keep the information.


+ **Update gate**:<br/>
Once we forget that the subject being discussed is singular, we need to find a way to update it to reflect the new subject is now plural. Here is the formula for the update gate.<br><br> 
$$\Gamma_{u}^{\langle t \rangle} = \sigma(W_{u}[a^{\langle t-1 \rangle}, x^{\langle t \rangle}] + b_{u})$$<br>
Similar to forget gate, update gate is vector of values between 0 and 1. This will be multiplied element-wise with $\tilde{c}^{\langle t \rangle}$, inorder to compute $c^{\langle t \rangle}$


+ **Updating the cell**:<br/>
To update the new subject we need to create a new vector of numbers that we can add to the previous cell state. The equation we use is:<br><br>
$$\tilde{c}^{\langle t \rangle} = \tanh(W_{c}[a^{\langle t-1 \rangle}, x^{\langle t \rangle}] + b_{c})$$<br>
Finally, the new cell state is:<br><br>
$$c^{\langle t \rangle} = \Gamma_{f}^{\langle t \rangle} \odot c^{\langle t-1 \rangle} + \Gamma_{u}^{\langle t \rangle} \odot \tilde{c}^{\langle t \rangle}$$<br>


+ **Output gate**:<br/>
To decide which ouputs we will use, we will use the following formula:<br><br>
$$\Gamma_{o}^{\langle t \rangle} = \sigma(W_{o}[a^{\langle t-1 \rangle}, x^{\langle t \rangle}] + b_{o})$$<br>
$$a^{\langle t \rangle} = \Gamma_{o}^{\langle t \rangle} \odot \tanh(c^{\langle t \rangle})$$<br>
First you decide what to output using sigmoid at the ouput gate and then you multiply that by $\tanh$ of the current cell state

### 2.1. LSTM cell

**Exercise**: Implement the LSTM cell described in the Figure (3).

**Instructions**:
1. Concatenate $a^{\langle t-1 \rangle}$ and $x^{\langle t \rangle}$ in a single matrix: $concat = \begin{bmatrix} a^{\langle t-1 \rangle} \\ x^{\langle t \rangle} \end{bmatrix}$
2. Compute all the formulas

In [9]:
def lstm_cell_forward(xt, a_prev, c_prev, parameters):
    """
    Implement a single forward step of the LSTM-cell as described in figure
    
    Argumets:
    ---------
    xt: np.array - (n_x, m), input data at time-step 't'
    a_prev: np.array - (n_a, m), hidden state at time-step 't-1' 
    c_prev: np.array - (n_a, m), memory state at time-step 't-1' 
    parameters: dict,
                Wf: np.array - (n_a, n_a + n_x), Weight matrix of the forget gate  
                bf: np.array - (n_a, 1), bias of the forget gate
                Wu: np.array - (n_a, n_a + n_x), weight matrix of the update gate 
                bu: np.array - (n_a, 1), bias of the update gate
                Wc: np.array - (n_a, n_a + n_x), weight matrix of the first 'tanh'
                bc: np.array - (n_a, 1), bias of the first 'tanh'
                Wo: np.array - (n_a, n_a + n_x), weight matrix of the output gate
                bo: np.array - (n_a, 1), bias of the output gate
                Wy: np.array - (n_y, n_a), weight matrix relating the hidden-state to the output
                by: np.array - (n_y, 1), bias relating the hidden-state to the ouput
    Returns:
    --------
    a_next: np.array - (n_a, n_m), next hidden state
    c_next: np.array - (n_a, n_m), next memory state
    yt_pred: np.array - (n_y, m), prediction at time-step 't'
    cache: tuple, values needed for backward pass, contains (a_next, c_next, a_prev, c_prev, xt, parameters)
                
    Note: ft/ut/ot stand stand for forget/update/output gates, cct stands for the candidate value (c tilde), 
          c stands for the memory value.
    """
    # retrieve parameters from parameters
    Wf = parameters['Wf']
    bf = parameters['bf']
    Wu = parameters['Wu']
    bu = parameters['bu']
    Wc = parameters['Wc']
    bc = parameters['bc']
    Wo = parameters['Wo']
    bo = parameters['bo']
    Wy = parameters['Wy']
    by = parameters['by']
    
    # retrieve dimensions from shapes of xt and Wy
    n_x, m = xt.shape
    n_y, n_a = Wy.shape
    
    # concatenate a_prev and xt
    concat = np.zeros((n_a + n_x, m))
    concat[:n_a, :] = a_prev
    concat[n_a:, :] = xt
    
    # compute values for ft, ut, cct, c_next, ot, a_next using the above formulas
    ft = sigmoid(np.dot(Wf, concat) + bf)
    ut = sigmoid(np.dot(Wu, concat) + bu)
    cct = np.tanh(np.dot(Wc, concat) + bc)
    c_next = ft * c_prev + ut * cct
    ot = sigmoid(np.dot(Wo, concat) + bo)
    a_next = ot * np.tanh(c_next)
    
    # compute prediction of the LSTM cell
    yt_pred = softmax(np.dot(Wy, a_next) + by)
    
    # store values needed for backward propagation in cache
    cache = (a_next, c_next, a_prev, c_prev, ft, ut, cct, ot, xt, parameters)
    
    return a_next, c_next, yt_pred, cache

In [12]:
np.random.seed(1)
xt = np.random.randn(3,10)
a_prev = np.random.randn(5,10)
c_prev = np.random.randn(5,10)
Wf = np.random.randn(5, 5+3)
bf = np.random.randn(5,1)
Wu = np.random.randn(5, 5+3)
bu = np.random.randn(5,1)
Wo = np.random.randn(5, 5+3)
bo = np.random.randn(5,1)
Wc = np.random.randn(5, 5+3)
bc = np.random.randn(5,1)
Wy = np.random.randn(2,5)
by = np.random.randn(2,1)

parameters = {"Wf": Wf, "Wu": Wu, "Wo": Wo, "Wc": Wc, "Wy": Wy, "bf": bf, "bu": bu, "bo": bo, "bc": bc, "by": by}

a_next, c_next, yt, cache = lstm_cell_forward(xt, a_prev, c_prev, parameters)
print("a_next[4] = ", a_next[4])
print("a_next.shape = ", c_next.shape)
print("c_next[2] = ", c_next[2])
print("c_next.shape = ", c_next.shape)
print("yt[1] =", yt[1])
print("yt.shape = ", yt.shape)
print("cache[1][3] =", cache[1][3])
print("len(cache) = ", len(cache))

a_next[4] =  [-0.66408471  0.0036921   0.02088357  0.22834167 -0.85575339  0.00138482
  0.76566531  0.34631421 -0.00215674  0.43827275]
a_next.shape =  (5, 10)
c_next[2] =  [ 0.63267805  1.00570849  0.35504474  0.20690913 -1.64566718  0.11832942
  0.76449811 -0.0981561  -0.74348425 -0.26810932]
c_next.shape =  (5, 10)
yt[1] = [0.79913913 0.15986619 0.22412122 0.15606108 0.97057211 0.31146381
 0.00943007 0.12666353 0.39380172 0.07828381]
yt.shape =  (2, 10)
cache[1][3] = [-0.16263996  1.03729328  0.72938082 -0.54101719  0.02752074 -0.30821874
  0.07651101 -1.03752894  1.41219977 -0.37647422]
len(cache) =  10


### 2.2. Forward pass for LSTM